In [1]:
import pandas as pd
import numpy as np
synthesized_data = pd.DataFrame(columns=[['sentence', 'v0', 'v1', 'v2', 'v3', 'v4','v5','v6'
                                         ,'v7','v8','v9']])
#for i in range(1000):
#    synthesized_data = pd.concat([synthesized_data,pd.DataFrame({'sentence': ('My name is pratik jayarao'),
                                                               
y_list = []
for i in range(1000):
    y_list.append([[1,0],[1,0],[1,0],[0,1],[0,1],[1,0],[1,0],[1,0],[0,1],[0,1]])

y_list = np.array(y_list)
print(y_list.shape)
#with open('person_name_git4.txt','a+') as my_file:
#    for i in range(len(synthesized_data)):,
#        my_file.write(synthesized_data.iloc[i]['sentence']+"\n")

import gensim
#import pandas as pd
#import numpy as np
from gensim.models.keyedvectors import KeyedVectors
#sentences = gensim.models.word2vec.LineSentence("person_name_git4.txt")
#print("sentences")
#model = gensim.models.word2vec.Word2Vec(sentences,sample = 0.001, sg = 1, workers = 4, negative = 20, window = 10, seed = 1, alpha = 0.025, min_count = 0, min_alpha = 0.0001, size = 100)
#print("model")
#model.save('person_name_vectors')



(1000, 10, 2)


In [2]:
word_vectors = gensim.models.Word2Vec.load('person_name_vectors')

In [3]:
wordVectors = []
for word in word_vectors.wv.index2word:
    wordVectors.append(word_vectors.wv[word])
wordVectors = np.array(wordVectors)
wordsList = word_vectors.wv.index2word


In [4]:
import tensorflow as tf
maxSeqLength = 10 #Maximum length of sentence
numDimensions = 100 #Dimensions for each word vector
firstSentence = np.zeros((maxSeqLength), dtype='int32')
firstSentence[0] = wordsList.index("pratik")
firstSentence[1] = wordsList.index("jayarao")
print(firstSentence.shape)
print(firstSentence)

(10,)
[3 4 0 0 0 0 0 0 0 0]


/home/pratik/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [5]:
with tf.Session() as sess:
    print(tf.nn.embedding_lookup(wordVectors,firstSentence).eval().shape)

train = synthesized_data

(10, 100)


In [6]:
#numFiles = len(train)
#maxSeqLength = 10
#ids = np.zeros((numFiles, maxSeqLength), dtype='int32')
#fileCounter = 0
#indexCounter = 0

#for line in train['sentence']:
#    cleanedLine = cleanSentences(str(line))
#    split = line.split()
#    indexCounter = 0
    
#    for word in split:
       # print(indexCounter)
#        try:
#            ids[fileCounter][indexCounter] = wordsList.index(word)
#        except ValueError:
#            ids[fileCounter][indexCounter] = len(wordsList)-2 #Vector for unkown words
#        indexCounter = indexCounter + 1
#        if indexCounter >= maxSeqLength:
#            break
#    fileCounter = fileCounter + 1 
#    print(fileCounter/numFiles)

In [7]:
#np.save('person_name_ids',ids)

In [8]:
ids = np.load('person_name_ids.npy')

In [9]:
ids.shape, train.shape, train[['v0', 'v1', 'v2', 'v3', 'v4','v5','v6'
                                         ,'v7','v8','v9']].shape

((1000, 10), (0, 11), (0, 10))

In [10]:
train

,sentence,v0,v1,v2,v3,v4,v5,v6,v7,v8,v9


In [11]:
batchSize = 100
lstmUnits = 64
numClasses = 2
epochs = 10
maxSeqLength = 10

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(ids,y_list, test_size=.2)

In [12]:
def getTrainBatch(j):
    i = j*batchSize
    arr = X_train[i:i+batchSize]
    labels = y_train[i:i+batchSize]
    
    return arr, labels

In [13]:
def getTestBatch(j):
    i = j*batchSize
    arr = X_test[i:i+batchSize]
    labels =   y_test[i:i+batchSize]
    return arr, labels

In [14]:
getTrainBatch(1)[1].shape

(100, 10, 2)

In [15]:
import tensorflow as tf
tf.reset_default_graph()
labels = tf.placeholder(tf.float32, [batchSize, maxSeqLength, numClasses])
input_data = tf.placeholder(tf.int32, [batchSize, maxSeqLength])

In [16]:
wordVectors = tf.convert_to_tensor(wordVectors, dtype=tf.float32)

In [17]:
data = tf.placeholder(shape = (batchSize,maxSeqLength,numDimensions), dtype=tf.float32)
data = tf.nn.embedding_lookup(wordVectors,input_data)


In [18]:
fw_cell3 = tf.nn.rnn_cell.MultiRNNCell([tf.nn.rnn_cell.BasicLSTMCell(lstmUnits, state_is_tuple=True) for _ in range(2)])
bw_cell3 = tf.nn.rnn_cell.MultiRNNCell([ tf.nn.rnn_cell.BasicLSTMCell(lstmUnits, state_is_tuple=True) for _ in range(2)])

In [19]:
outputs,value2 = tf.nn.bidirectional_dynamic_rnn(fw_cell3, bw_cell3,data,dtype=tf.float32)

In [20]:
outputs = tf.concat(outputs, 2)

In [21]:
weight = tf.Variable(tf.truncated_normal([lstmUnits*2,numClasses]))
bias = tf.Variable(tf.constant(0.1, shape=[batchSize,maxSeqLength,numClasses]))

In [22]:
prediction = (tf.tensordot(outputs, weight, axes=((2,),(0,))) + bias)
correctPred = tf.equal(tf.argmax(prediction,axis=2), tf.argmax(labels,axis=2))
accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))


In [23]:
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=prediction, labels=labels))
optimizer = tf.train.AdamOptimizer().minimize(loss)

In [24]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    o,w,b,p = sess.run([outputs,weight,bias,prediction],feed_dict={input_data: X_train[:100], labels: y_train[:100]})
    print(np.array(o).shape,np.array(w).shape,np.array(b).shape,np.array(p).shape)

(100, 10, 128) (128, 2) (100, 10, 2) (100, 10, 2)


In [25]:
import datetime

tf.summary.scalar('Loss', loss)
tf.summary.scalar('Accuracy', accuracy)
merged = tf.summary.merge_all()
logdir = "tensorboard/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + "/"
writer = tf.summary.FileWriter(logdir, sess.graph)

In [26]:
sess = tf.InteractiveSession()
saver = tf.train.Saver()
sess.run(tf.global_variables_initializer())

for i in range(epochs):
    print("epoch:  "+str(i))
    #Next Batch of reviews
    for j in range(len(X_train)//batchSize):
        nextBatch, nextBatchLabels = getTrainBatch(j);
        #print(j/2)
        _,pred,lab = sess.run([optimizer,prediction,labels], {input_data: nextBatch, labels: nextBatchLabels})

    #Write summary to Tensorboard
        if (j % 5 == 0):
            summary = sess.run(merged, {input_data: nextBatch, labels: nextBatchLabels})
            writer.add_summary(summary, j)

        #Save the network every 10,000 training iterations
        if (j % 50 == 0 and j != 0):
            save_path = saver.save(sess, "models/pretrained_lstm.ckpt", global_step=i)
            print("saved to %s" % save_path)
print((np.array(pred)))
x3 = np.array(lab)
writer.close()

epoch:  0
epoch:  1
epoch:  2
epoch:  3
epoch:  4
epoch:  5
epoch:  6
epoch:  7
epoch:  8
epoch:  9
[[[ 14.949078  -13.304953 ]
  [ 10.696237  -10.12192  ]
  [  3.3658986  -3.5698216]
  ...
  [  4.0793324  -4.1829925]
  [ -3.920346    3.7513685]
  [-11.193136   10.928375 ]]

 [[ 14.949078  -13.304953 ]
  [ 10.696237  -10.12192  ]
  [  3.3658986  -3.5698216]
  ...
  [  4.0793324  -4.1829925]
  [ -3.920346    3.7513685]
  [-11.193136   10.928375 ]]

 [[ 14.949078  -13.304953 ]
  [ 10.696237  -10.12192  ]
  [  3.3658986  -3.5698216]
  ...
  [  4.0793324  -4.1829925]
  [ -3.920346    3.7513685]
  [-11.193136   10.928375 ]]

 ...

 [[ 14.949078  -13.304953 ]
  [ 10.696237  -10.12192  ]
  [  3.3658986  -3.5698216]
  ...
  [  4.0793324  -4.1829925]
  [ -3.920346    3.7513685]
  [-11.193136   10.928375 ]]

 [[ 14.949078  -13.304953 ]
  [ 10.696237  -10.12192  ]
  [  3.3658986  -3.5698216]
  ...
  [  4.0793324  -4.1829925]
  [ -3.920346    3.7513685]
  [-11.193136   10.928375 ]]

 [[ 14.949078 

In [31]:
test_batches = 2
for j in range(test_batches):
    nextBatch, nextBatchLabels = getTestBatch(j);
    print("Accuracy for this batch:", (sess.run(accuracy, {input_data: nextBatch, labels: nextBatchLabels})) * 100)

Accuracy for this batch: 100.0
Accuracy for this batch: 100.0


In [28]:
x = np.array([[ 15.984614 , -18.476969 ],
 [ 11.581611 , -13.3633375],
 [  4.762972 ,  -5.3671947],
 [ -3.291655 ,   3.5226593],
 [ -3.8651543 ,  3.9896889],
 [  4.91325   , -4.6390204],
 [  9.824387  , -9.47608  ],
 [  5.9091616 , -5.683087 ],
 [ -3.9726372 ,  3.8681452],
 [-12.435596  , 12.365538 ]])

In [29]:
x3.argmax(axis = 2)

array([[0, 0, 0, 1, 1, 0, 0, 0, 1, 1],
       [0, 0, 0, 1, 1, 0, 0, 0, 1, 1],
       [0, 0, 0, 1, 1, 0, 0, 0, 1, 1],
       [0, 0, 0, 1, 1, 0, 0, 0, 1, 1],
       [0, 0, 0, 1, 1, 0, 0, 0, 1, 1],
       [0, 0, 0, 1, 1, 0, 0, 0, 1, 1],
       [0, 0, 0, 1, 1, 0, 0, 0, 1, 1],
       [0, 0, 0, 1, 1, 0, 0, 0, 1, 1],
       [0, 0, 0, 1, 1, 0, 0, 0, 1, 1],
       [0, 0, 0, 1, 1, 0, 0, 0, 1, 1],
       [0, 0, 0, 1, 1, 0, 0, 0, 1, 1],
       [0, 0, 0, 1, 1, 0, 0, 0, 1, 1],
       [0, 0, 0, 1, 1, 0, 0, 0, 1, 1],
       [0, 0, 0, 1, 1, 0, 0, 0, 1, 1],
       [0, 0, 0, 1, 1, 0, 0, 0, 1, 1],
       [0, 0, 0, 1, 1, 0, 0, 0, 1, 1],
       [0, 0, 0, 1, 1, 0, 0, 0, 1, 1],
       [0, 0, 0, 1, 1, 0, 0, 0, 1, 1],
       [0, 0, 0, 1, 1, 0, 0, 0, 1, 1],
       [0, 0, 0, 1, 1, 0, 0, 0, 1, 1],
       [0, 0, 0, 1, 1, 0, 0, 0, 1, 1],
       [0, 0, 0, 1, 1, 0, 0, 0, 1, 1],
       [0, 0, 0, 1, 1, 0, 0, 0, 1, 1],
       [0, 0, 0, 1, 1, 0, 0, 0, 1, 1],
       [0, 0, 0, 1, 1, 0, 0, 0, 1, 1],
       [0, 0, 0, 1, 1, 0,

In [30]:
x2.argmax(axis=2)

NameError: name 'x2' is not defined